In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import cross_validation as cv


In [3]:
df = pd.read_csv("./../hw1/spam.train.txt", delim_whitespace=True, header=None)
x_data = df.ix[:, 1:].as_matrix(); y_data = df.ix[:, 0].as_matrix()
y_data[y_data == 0] = -1
x_train, x_test, y_train, y_test = cv.train_test_split(x_data, y_data,
                                                       test_size=0.25, random_state=288)


# CFS

In [4]:
from src.cfs import cfs

In [25]:
cfs_features = cfs(x_train, y_train, 1000)

In [26]:
cfs_features

[48,
 8,
 70,
 10,
 49,
 101,
 53,
 52,
 76,
 32,
 31,
 75,
 73,
 19,
 56,
 4,
 80,
 9,
 100,
 35,
 71,
 86,
 61,
 41,
 72]

# Wrapper

In [5]:
from sklearn.metrics import f1_score

In [6]:
from sklearn.ensemble import AdaBoostClassifier

In [7]:
abc = AdaBoostClassifier(n_estimators=50, learning_rate=0.2)
abc.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.2, n_estimators=50, random_state=None)

In [8]:
f1_score(y_test, abc.predict(x_test))

0.95656417764763302

In [19]:
from joblib import Parallel, delayed

In [20]:
def pred_(x_train, y_train):
    abc = AdaBoostClassifier(n_estimators=50, learning_rate=0.2)
    abc.fit(x_train, y_train)
    return abc

In [23]:
def wrapper(expected_features, x_train, y_train, x_test, y_test):
    initial_fset = set(np.arange(x_train.shape[1]))
    target_flist = list()

    for i in range(expected_features+1):
        target_fset = set(target_flist)
        rest_fset = list(initial_fset - target_fset)
        
        unbound_fset = [list(target_fset|set([f])) for f in rest_fset]
        clfs = Parallel(n_jobs=8)(delayed(pred_)(x_train[:, fset], y_train) for fset in unbound_fset)

        clfs_scores = [f1_score(y_test, clf.predict(x_test[:, unbound_fset[i]])) for i, clf in enumerate(clfs)]
        best_f_ind = np.argmax(clfs_scores)

        best_f = rest_fset[best_f_ind]
        target_flist.append(best_f)
        print target_flist

    return target_fset

wrapper_features = wrapper(60, x_train, y_train, x_test, y_test)

[48]
[48, 53]
[48, 53, 35]
[48, 53, 35, 41]
[48, 53, 35, 41, 71]
[48, 53, 35, 41, 71, 4]
[48, 53, 35, 41, 71, 4, 83]
[48, 53, 35, 41, 71, 4, 83, 70]
[48, 53, 35, 41, 71, 4, 83, 70, 56]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73, 31]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73, 31, 100]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73, 31, 100, 77]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73, 31, 100, 77, 91]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73, 31, 100, 77, 91, 0]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73, 31, 100, 77, 91, 0, 1]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73, 31, 100, 77, 91, 0, 1, 2]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73, 31, 100, 77, 91, 0, 1, 2, 5]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73, 31, 100, 77, 91, 0, 1, 2, 5, 6]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73, 31, 100, 77, 91, 0, 1, 2, 5, 6, 7]
[48, 53, 35, 41, 71, 4, 83, 70, 56, 52, 73, 31, 100, 77, 91, 0, 1, 2, 5, 6, 7, 12]


# Embedded

In [ ]:
f_w = np.zeros(x_train.shape[1])

def wide_walk(tree, feature_weigths):
    if tree.split_feature is None:
        return feature_weigths
    
    feature_weigths[tree.split_feature] += tree.score
    
    feature_weigths = wide_walk(tree.fb, feature_weigths)
    feature_weigths = wide_walk(tree.tb, feature_weigths)
    
    return feature_weigths
wide_walk(gboost.ensembles[3].tree, f_w)
#np.sum(np.array([wide_walk(p.tree, np.zeros(x_train.shape[1])) for p in predictors[0].ensembles]), axis=0)